In [1]:
import scipy.io
import numpy as np
import CMR_IA as cmr
import matplotlib.pyplot as plt
import pickle
from statistics import mean, stdev
import pprint
# import import_ipynb
import time
import pandas as pd
import seaborn as sns

np.set_printoptions(edgeitems=20, linewidth=150,precision=4)

In [2]:
with open("../Data/simu7_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)

In [3]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [4]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.5,
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0.01,
    eta = 0,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 10,
    d_ass = 1,
    learn_while_retrieving = True
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.5,
 'beta_rec_new': 0.5,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0,
 'omega': 8,
 'alpha': 4,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.01,
 'd_ass': 1,
 'lamb': 0.01,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 10,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [5]:
df_study = df_study.loc[df_study.session == 0]
# df_study['study_itemno1'] = [1,3,5,7,9,11,13,15,17,19,21,23]
# df_study['study_itemno2'] = [2,4,6,8,10,12,14,16,18,20,22,24]
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,serial_pos,list,session
0,110,1083,BED,PLANT,0,0,0
1,1262,1627,SEAL,WRITER,1,0,0
2,738,1198,INMATE,ROBE,2,0,0
3,187,748,BUG,JAM,3,0,0
4,1423,1072,SWORD,PIN,4,0,0
...,...,...,...,...,...,...,...
187,1068,485,PILL,DONOR,7,15,0
188,490,1537,DOUGHNUT,UNDERWEAR,8,15,0
189,977,323,ONION,CLIMBER,9,15,0
190,1148,825,QUARTER,LINEN,10,15,0


In [6]:
df_test = df_test.loc[df_test.session == 0]
# df_test['test_itemno'] = [1]
# df_test['correct_ans'] = [2]
df_test

,test_itemno,test_item,correct_ans,study_pos,test_dir,list,session
0,324,CLIPPERS,823,9,1,0,0
1,1347,SPARROW,1051,8,1,0,0
2,413,CROSS,524,7,0,0,0
3,812,LEOPARD,1323,5,0,0,0
4,110,BED,1083,0,0,0,0
...,...,...,...,...,...,...,...
123,693,HAY,10,4,0,15,0
124,1537,UNDERWEAR,490,8,1,15,0
125,825,LINEN,1148,10,1,15,0
126,1306,SIGNATURE,179,0,0,15,0


In [7]:
df_thin = df_test[['list','session','test_itemno']]
df_thin = df_thin.assign(s_resp=np.nan, s_rt=np.nan)
df_thin

,list,session,test_itemno,s_resp,s_rt
0,0,0,324,NaN,NaN
1,0,0,1347,NaN,NaN
2,0,0,413,NaN,NaN
3,0,0,812,NaN,NaN
4,0,0,110,NaN,NaN
...,...,...,...,...,...
123,15,0,693,NaN,NaN
124,15,0,1537,NaN,NaN
125,15,0,825,NaN,NaN
126,15,0,1306,NaN,NaN


In [8]:
# extarct the session data
sess = 0
list_num = len(np.unique(df_study.list))

pres_mat = df_study.loc[df_study.session == sess, ['study_itemno1', 'study_itemno2']].to_numpy()
pres_mat = np.reshape(pres_mat, (list_num, -1, 2))
pres_mat

array([[[ 110, 1083],
        [1262, 1627],
        [ 738, 1198],
        [ 187,  748],
        [1423, 1072],
        [ 812, 1323],
        [ 679,  879],
        [ 413,  524],
        [1051, 1347],
        [ 823,  324],
        [ 152,  680],
        [ 824,  841]],

       [[1119, 1628],
        [1183, 1256],
        [1093,   35],
        [ 666,  456],
        [1446,  586],
        [ 396,  844],
        [ 999, 1292],
        [1102, 1021],
        [1590,  528],
        [1540,  232],
        [1248, 1141],
        [ 506, 1534]],

       [[ 920,  605],
        [1636,  688],
        [  34, 1016],
        [ 142, 1399],
        [ 570, 1444],
        [1055,  505],
        [ 716,  113],
        [1523, 1117],
        [1219, 1209],
        [1002, 1465],
        [ 173,  414],
        [ 492,  845]],

       [[1069, 1608],
        [ 429,  379],
        [1437, 1508],
        [ 650,   30],
        [ 300,  157],
        [ 734,  911],
        [ 245, 1410],
        [ 744,  493],
        [1115,  127],
    

In [9]:
cue_mat = df_thin.loc[df_thin.session == sess, 'test_itemno'].to_numpy()
cue_mat = np.reshape(cue_mat,(list_num,-1))
cue_mat

array([[ 324, 1347,  413,  812,  110, 1198,  879,  152],
       [1141, 1540,  528,  506,  844,  586, 1093,  666],
       [ 173,  492, 1465,  605, 1399, 1523, 1219,  505],
       [ 611,  621, 1508,  245, 1115,  659,  734, 1608],
       [1185,  259,  988,  203,  753,  577,  553, 1569],
       [  52,  673,  717,  358, 1366,  290,  453, 1158],
       [ 638,  732,  265,  170,  420,  923, 1000,  476],
       [ 154, 1203,  645,  589,  163, 1043, 1129,  129],
       [1067, 1291,  795,  231,  882, 1053,  884,  120],
       [ 546,  284,  390,  939,    9,   89, 1331,  395],
       [1625, 1076, 1552,  542, 1153,  401,  303,  912],
       [1319,   82,   16,  431,  494, 1462, 1500,  890],
       [ 263, 1515,  147,  652,  189,  364,  293,  719],
       [1428, 1200, 1389,  714,  784,  866,  787,  272],
       [ 210,  938,  483,  640,  700, 1254, 1082,  465],
       [1609,  485, 1445,  693, 1537,  825, 1306,  653]])

In [10]:
sem_mat = s_mat

In [11]:
model = cmr.CMR2(params, pres_mat, sem_mat, source_mat=None,
           rec_mat=None, ffr_mat=None, cue_mat=cue_mat, task='CR', mode='Final')

In [12]:
model.pres_nos

array([[[ 110, 1083],
        [1262, 1627],
        [ 738, 1198],
        [ 187,  748],
        [1423, 1072],
        [ 812, 1323],
        [ 679,  879],
        [ 413,  524],
        [1051, 1347],
        [ 823,  324],
        [ 152,  680],
        [ 824,  841]],

       [[1119, 1628],
        [1183, 1256],
        [1093,   35],
        [ 666,  456],
        [1446,  586],
        [ 396,  844],
        [ 999, 1292],
        [1102, 1021],
        [1590,  528],
        [1540,  232],
        [1248, 1141],
        [ 506, 1534]],

       [[ 920,  605],
        [1636,  688],
        [  34, 1016],
        [ 142, 1399],
        [ 570, 1444],
        [1055,  505],
        [ 716,  113],
        [1523, 1117],
        [1219, 1209],
        [1002, 1465],
        [ 173,  414],
        [ 492,  845]],

       [[1069, 1608],
        [ 429,  379],
        [1437, 1508],
        [ 650,   30],
        [ 300,  157],
        [ 734,  911],
        [ 245, 1410],
        [ 744,  493],
        [1115,  127],
    

In [13]:
model.cues_indexes

array([[ 74, 325,  92, 197,  21, 300, 214,  33],
       [287, 367, 127, 121, 205, 141, 277, 164],
       [ 40, 116, 348, 144, 331, 363, 308, 120],
       [146, 149, 358,  58, 281, 161, 181, 376],
       [297,  62, 244,  47, 185, 137, 132, 370],
       [ 10, 166, 176,  78, 326,  69, 104, 293],
       [154, 180,  64,  38,  94, 229, 251, 111],
       [ 34, 303, 157, 142,  37, 261, 284,  29],
       [267, 317, 195,  53, 216, 263, 217,  25],
       [130,  68,  87, 233,   0,  17, 324,  88],
       [380, 273, 368, 129, 291,  90,  72, 226],
       [321,  15,   2,  98, 118, 347, 356, 218],
       [ 63, 360,  32, 159,  44,  80,  70, 177],
       [337, 301, 328, 173, 193, 211, 194,  67],
       [ 48, 232, 113, 155, 172, 311, 274, 108],
       [377, 114, 343, 171, 366, 201, 319, 160]], dtype=int64)

In [14]:
model.pres_indexes

array([[[ 21, 275],
        [313, 381],
        [182, 300],
        [ 42, 184],
        [336, 272],
        [197, 322],
        [167, 214],
        [ 92, 126],
        [262, 325],
        [199,  74],
        [ 33, 168],
        [200, 204]],

       [[283, 382],
        [296, 312],
        [277,   6],
        [164, 105],
        [344, 141],
        [ 89, 205],
        [250, 318],
        [278, 255],
        [374, 127],
        [367,  54],
        [310, 287],
        [121, 365]],

       [[228, 144],
        [383, 170],
        [  5, 254],
        [ 31, 331],
        [134, 342],
        [264, 120],
        [175,  22],
        [363, 282],
        [308, 304],
        [252, 348],
        [ 40,  93],
        [116, 206]],

       [[269, 376],
        [ 97,  82],
        [338, 358],
        [158,   3],
        [ 71,  35],
        [181, 225],
        [ 58, 333],
        [183, 117],
        [281,  27],
        [236, 146],
        [302, 149],
        [161,  65]],

       [[137, 125],
        [237

In [15]:
model.nlists

16

In [16]:
model.run_intrusion_cr_single_sess()

# in model:
# print(self.att_vec[self.all_nos_unique[item_idx]-1])
# print(np.dot(self.c_old[:self.nitems_unique], self.f[:self.nitems_unique].T))

In [ ]:
model.ret_thresh

In [ ]:
model.rec_items

In [ ]:
model.rec_times

In [ ]:
model.f_in

In [ ]:
model.M_FC

In [ ]:
model.M_CF

In [ ]:
recs = model.rec_items
rts = model.rec_times
result = np.column_stack((recs,rts))
df_thin.loc[df_thin.session==sess, ['s_resp','s_rt']] = result
df_simu = df_thin.merge(df_test,on=['session','test_itemno'])
df_simu

In [ ]:
tmp1 = df_study.study_itemno1.to_numpy()
tmp2 = df_study.study_itemno2.to_numpy()
tmp = np.concatenate((tmp1, tmp2))
tmp = np.sort(tmp)
tmp

In [ ]:
testid = np.searchsorted(tmp, df_simu.test_itemno.item())
corrid = np.searchsorted(tmp, df_simu.correct_ans.item())

In [ ]:
df_simu['corr_fin']= model.f_in[corrid]
df_simu['othe_fin'] = np.mean(np.delete(model.f_in, [testid,corrid]))
df_simu